# 1. Run the cell below to load the function. 

In part 2 you can test different values.

In [11]:
import pandas as pd
from datetime import datetime
# Display DataFrame without wrapping the cols
pd.set_option('display.max_columns', None) # Set to None to display all columns
pd.set_option('display.width', 1000) # Set width to a high value to prevent wrapping
pd.options.mode.chained_assignment = None  # Disable the warning

# Please, specify path to the CDPOS file
path = 'datasets/CDPOS.csv'

def calculate_net_order_value(MANDT, EBELN, D_DATE):
    """
    Calculate the net order value for a specific purchase order on a given date.

    Parameters:
        MANDT (str): Unique client ID, e.g. "010".
        EBELN (str): Purchase order number, e.g. "4700001106".
        D_DATE (str): Date in 'YYYY-MM-DD' format.

    Returns:
        float: Total net order value for the specified purchase order and date.
        pandas.DataFrame: Filtered DataFrame containing the records up until the specified D_DATE.
    """
    df = pd.read_csv(path, dtype='object')

    # Dropping missing values
    df = df.dropna(subset = ['VALUE_NEW'])

    # Stripping all spaces from 'VALUE_NEW' column
    df.loc[:, 'VALUE_NEW'] = df['VALUE_NEW'].str.strip()

    # Dropping all rows that contain letters in 'VALUE_NEW' column
    df = df[~df['VALUE_NEW'].str.contains('[a-zA-Z]')]

    # Convert VALUE_NEW into float
    df['VALUE_NEW'] = df['VALUE_NEW'].astype(float)

    # Convert 'UDATE' column to datetime data type
    df['UDATE'] = pd.to_datetime(df['UDATE']).dt.strftime('%Y-%m-%d')

    # Convert 'UTIME' to datetime and to format 'THH:MM:SS'
    df['UTIME'] = pd.to_datetime(df['UTIME']).dt.strftime('T%H:%M:%S')

    # concat UDATE and UTIME into a new column called D_DATE
    df['D_DATE'] = df['UDATE'] + '' + df['UTIME']

    # Convert D_DATE to datetime
    df['D_DATE'] = pd.to_datetime(df['D_DATE'])

    # Creating new column from the 5 last digits and assigning it as 4th column
    df.insert(3, 'EBELP', df['TABKEY'].str[-5:])

    # Filtering FNAME
    df = df[df['FNAME'] == 'NETWR']

    # Renaming columns for the function
    df = df.rename(columns={'MANDANT': 'MANDT', 'OBJECTID': 'EBELN', 'VALUE_NEW': 'NETWR'}).reset_index(drop=True)

    # Dropping columns
    df = df.drop(['OBJECTCLAS', 'CHANGENR', 'TABKEY', 'CHNGIND', 'UTIME', 'UDATE', 'TABNAME'], axis=1)

    # Filtering MANDT and EBELN specified by the user
    filtered_df = df[(df['MANDT'] == MANDT) & (df['EBELN'] == EBELN)]

    # Convert D_DATE to datetime
    date_value = datetime.strptime(D_DATE, '%Y-%m-%d')
    filtered_df['D_DATE'] = pd.to_datetime(filtered_df['D_DATE'])

    # Filter records up until the specified D_DATE
    filtered_df = filtered_df[filtered_df['D_DATE'] <= date_value]

    # Keep only the most recent record for each EBELP
    filtered_df = filtered_df.sort_values(by='D_DATE', ascending=False)
    filtered_df = filtered_df.drop_duplicates(subset='EBELP', keep='first') # By commenting this out, all the records untill given date will be displayed and summed
    
    net_order_value = filtered_df['NETWR'].sum()

    return net_order_value, filtered_df

# 2. User Input

In [12]:
# Please, specify desired input parameters
MANDT = "010"
EBELN = "4700001106"
D_DATE = "2018-01-30" # The format must be YYYY-MM-DD (e.g. 2018-01-30)

# 3. Run the cell below.

Result displayed under the cell below.

In [13]:
result, filtered_records = calculate_net_order_value(MANDT, EBELN, D_DATE)
if result is not None:
    print(filtered_records)
    print()
    print(f"Total net order value for MANDT {MANDT}, EBELN {EBELN}, up until D_DATE {D_DATE}: {result}")
else:
    print("Invalid MANDT/EBELN/D_DATE format")

  MANDT       EBELN  EBELP  FNAME    NETWR       VALUE_OLD              D_DATE
8   010  4700001106  00010  NETWR   2500.0         1332.49 2018-01-22 10:10:00
2   010  4700001106  00030  NETWR  25000.0        50000.00 2018-01-09 09:24:08

Total net order value for MANDT 010, EBELN 4700001106, up until D_DATE 2018-01-30: 27500.0


# 4. Test different input:

In [14]:
# Please, specify desired input parameters
MANDT = "010"
EBELN = "4700001106"
D_DATE = "2018-01-10" # The format must be YYYY-MM-DD (e.g. 2018-01-30)

result, filtered_records = calculate_net_order_value(MANDT, EBELN, D_DATE)
if result is not None:
    print(filtered_records)
    print()
    print(f"Total net order value for MANDT {MANDT}, EBELN {EBELN}, up until D_DATE {D_DATE}: {result}")
else:
    print("Invalid MANDT/EBELN/D_DATE format")

  MANDT       EBELN  EBELP  FNAME    NETWR       VALUE_OLD              D_DATE
2   010  4700001106  00030  NETWR  25000.0        50000.00 2018-01-09 09:24:08

Total net order value for MANDT 010, EBELN 4700001106, up until D_DATE 2018-01-10: 25000.0
